In [1]:
# https://freeprog.tistory.com/329
#update 7 버전에서는 이미 있는 엑셀의 수정이 용이하도록 순서에 따른 로직이 아니라 리스트 검색에 따른 로직으로 작성함.
#sim2 버전에서는 pit 형상 판별없이빠르게  깊이 값만 측정하도록 하며,
#동일한 이름의 엑셀파일은 동시에 열 수 없기 때문에 다른이름으로 저장함 . 
#UP9_1버전에서는 이미지 스크린샷 후 저장기능을  추가함.
#UP9_3버전에서는 저장할이미지에  텍스트 추가.
#UP9_4버전에서는 Pit shape 측정 추가.
#UP9_5버전에서는 엑셀에 Pit Shape 값 저장. pen 선택을 없애고 pen을 고정 .
#UP9_6버전에서는 Pit shape 측정의 undercutting(e) type 의 angle값 수정.버림을 반올림으로 변경.
#restart_y 에러 해결. 화면바뀔 때 itnum 초기화.
#UP9_7버전에서는 drawtext의 글자색 검정색, undercutting(e) type 의 drawline 수정. 사진에 depth 안나타나는 오류해결.  
#UP9_7버전에서 UP9_6부터 생긴 angle 이상값 수정. pit type (a,b,c,e) 판별 후  출력. 각종에러해결.   전체화면.
#UP9_8버전에서는 최적화. global assignment 오류해결. x와 ?를 구별하여 엑셀 저장. ?는 pit depth까지만 측정.
#UP9_9버전에서는 실제 test.C보다 wide 한 CC추가. (d)Subsurface 추가
#UP9_10버전에서는 엑셀의 첫 행을 이름으로함 .
#UP9_10_yellow버전에서는 선 색을 노란색으로함. 

import sys
import IPython
import imutils
import cv2
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *

from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse

import xlwings as xw
import os
import subprocess
from parse import *

QApplication.setAttribute(Qt.AA_EnableHighDpiScaling, True)



class Left(QWidget): 
    from PIL import Image,ImageFont,ImageDraw 
    w= 0
    h= 0
    pixelsPerMetric=0
    fname= ''
    finame = ''
    sflist = []
    dir1= ''
    num = 0
    i= 0

    dB0=0
    exnum = 0
    exshadow=[]
    width = 10
    xl_updated_last=""
    global comment
    comment =""

    def __init__(self):
    
        super().__init__()
           
#         astm=cv2.imread('C:/Users/Hojin/Anaconda3/envs/tensorflow/MYY/Measure/mouse_tracking/images/astm_pit.jpg')
#         cv2.imshow("ASTM",astm )
#         # cv2.waitKey(0)

        
        # 전체 폼 박스
        self.formbox = QHBoxLayout()
        self.setLayout(self.formbox)

        # 좌, 우 레이아웃박스
        self.left = QVBoxLayout()
        self.middle = QVBoxLayout()

        # 그룹박스1 생성 및 좌 레이아웃 배치
        gb = QGroupBox('그리기 종류')        
        self.left.addWidget(gb)

        # 그룹박스1 에서 사용할 레이아웃
        box = QVBoxLayout()
        gb.setLayout(box)        

#         # 그룹박스 1 의 라디오 버튼 배치
#         text = ['horizon']
#         self.radiobtns = []

#         for i in range(len(text)):
#             self.radiobtns.append(QRadioButton(text[i], self))
#             self.radiobtns[i].clicked.connect(self.radioClicked)
#             box.addWidget(self.radiobtns[i])

#         self.radiobtns[0].setChecked(True)
        self.drawType = 0
         
#         # 그룹박스2
#         gb = QGroupBox('펜 설정')        
#         self.left.addWidget(gb)        

#         grid = QGridLayout()      
#         gb.setLayout(grid)        

#         label = QLabel('선굵기')
#         grid.addWidget(label, 0, 0)

#         self.combo = QComboBox()
#         grid.addWidget(self.combo, 0, 1)       

#         for i in range(1, 21):
#             self.combo.addItem(str(i))

#         label = QLabel('선색상')

#         grid.addWidget(label, 1,0)        
         
#         self.pencolor = QColor(0,0,0)
#         self.penbtn = QPushButton()        
#         self.penbtn.setStyleSheet('background-color: rgb(0,0,0)')
#         self.penbtn.clicked.connect(self.showColorDlg)
#         grid.addWidget(self.penbtn,1, 1)
         

#         # 그룹박스3
#         gb = QGroupBox('붓 설정')        
#         self.left.addWidget(gb)
#         hbox = QHBoxLayout()
#         gb.setLayout(hbox)

#         label = QLabel('붓색상')
#         hbox.addWidget(label)                

#         self.brushcolor = QColor(255,255,255)
#         self.brushbtn = QPushButton()        
#         self.brushbtn.setStyleSheet('background-color: rgb(255,255,255)')
#         self.brushbtn.clicked.connect(self.showColorDlg)
#         hbox.addWidget(self.brushbtn)
        
        
        # 그룹박스4
        gb = QGroupBox('불러오기')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)    
            
        self.openbtn = QPushButton("File Open")
        self.openbtn.clicked.connect(self.openbtnClicked)
        self.label = QLabel()

        hbox.addWidget(self.openbtn)
        hbox.addWidget(self.label)
        
         # 그룹박스5
        gb = QGroupBox('불러오기 방법')        
        self.left.addWidget(gb)

        box2 = QHBoxLayout()
        gb.setLayout(box2)        

        text = ['*.확장자(unstack)', '개별(unstack)']
        self.radiobtns2 = []

        for i in range(len(text)):
            self.radiobtns2.append(QRadioButton(text[i], self))
            self.radiobtns2[i].clicked.connect(self.radio2Clicked)
            box2.addWidget(self.radiobtns2[i])

        self.radiobtns2[0].setChecked(True)
        self.loadType = 0
         
    
        #그룹박스 6
        gb=QGroupBox('처리할 목록')
        self.left.addWidget(gb)
        
        grid2= QGridLayout()
        gb.setLayout(grid2)
        
        
        self.tb = QTextBrowser()
        self.tb.setAcceptRichText(True)
        self.tb.setOpenExternalLinks(True)

        self.previous_btn = QPushButton('previous')
        self.previous_btn.pressed.connect(self.previous_view)
        self.next_btn = QPushButton('Next')
        self.next_btn.pressed.connect(self.next_view)
        self.clear_btn = QPushButton('Clear')
        self.clear_btn.pressed.connect(self.clear_text)

        self.lbl_000 = QLabel("  ")
        
        grid2.addWidget(self.lbl_000,0,0)
        grid2.addWidget(self.tb, 1,0)
        grid2.addWidget(self.clear_btn, 2,0)
        grid2.addWidget(self.previous_btn,3,0)
        grid2.addWidget(self.next_btn, 4,0)
        
        
        # 그룹박스7
        gb = QGroupBox('스케일 바')        
        
        self.left.addWidget(gb)
        grid3=QGridLayout()
        gb.setLayout(grid3)
        # LineEdit
        self.lineEdit = QLineEdit("", self)
#         self.lineEdit.returnPressed.connect(self.lineEditChanged)
        self.lineEdit.textChanged.connect(self.lineEditChanged)
        grid3.addWidget(self.lineEdit,0,0)
        self.statusBar = QStatusBar(self)
        
#         self.lbl_bar=QLabel("  ")
        grid3.addWidget(self.statusBar,0,1)
        
        
        
        # 그룹박스7
        gb = QGroupBox('스케일')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)
        
        self.scalebtn = QPushButton()  
        self.scalebtn.clicked.connect(self.scalepnt)
        
        hbox.addWidget(self.scalebtn)
        self.scalebtn.setText('print')
        
        self.lbl_1 = QLabel("  ")
        hbox.addWidget(self.lbl_1)
        
        
        # 그룹박스8
        gb = QGroupBox('크기 측정')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)
        
        self.pixelpntbtn = QPushButton()  
        self.pixelpntbtn.clicked.connect(self.pixelpnt)
        
        hbox.addWidget(self.pixelpntbtn)
        self.pixelpntbtn.setText('print')
        
        self.lbl_2 = QLabel("  ")
        hbox.addWidget(self.lbl_2)
        
        
         # 그룹박스9
        gb = QGroupBox('엑셀')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)
        
        self.excelbtn = QPushButton()  
        self.excelbtn.clicked.connect(self.excelup)
        
        hbox.addWidget(self.excelbtn)
        self.excelbtn.setText('엑셀업댓')
        
        self.lbl_3 = QLabel("  ")
        hbox.addWidget(self.lbl_3)
        
        
         # 그룹박스10
        gb = QGroupBox('스크린샷')        
        self.left.addWidget(gb)
        hbox = QHBoxLayout()
        gb.setLayout(hbox)
        
        self.scrshbtn = QPushButton()  
        self.scrshbtn.clicked.connect(self.take_screenshot)
        
        hbox.addWidget(self.scrshbtn)
        self.scrshbtn.setText('screenshot')
        
        self.lbl_4 = QLabel("  ")
        hbox.addWidget(self.lbl_4)
        
        self.left.addStretch(1)
        
        
        
        # 전체 폼박스에 왼쪽 박스 배치
        self.formbox.addLayout(self.left)
        self.formbox.setStretchFactor(self.left, 0)

        self.openbtnClicked()#############파일 선택
        
    def combo_d_c(self):
        self.cd=self.combo_d.currentText()
           
    def combo_d2_c(self):
        self.cd2=self.combo_d2.currentText()
        
    def radioClicked(self):
        for i in range(len(self.radiobtns)):
            if self.radiobtns[i].isChecked():
                self.drawType = i                
                break

    def checkClicked(self):
        pass
             
    def showColorDlg(self):       
         
        # 색상 대화상자 생성      
        color = QColorDialog.getColor()

        sender = self.sender()

        # 색상이 유효한 값이면 참, QFrame에 색 적용
        if sender == self.penbtn and color.isValid():           
            self.pencolor = color
            self.penbtn.setStyleSheet('background-color: {}'.format( color.name()))
        else:
            self.brushcolor = color
            self.brushbtn.setStyleSheet('background-color: {}'.format( color.name()))

            

    def clear_text(self) :

        self.tb.clear()

        
    def radio2Clicked(self):
        for i in range(len(self.radiobtns2)):
            if self.radiobtns2[i].isChecked():
                self.loadType = i                
                break
        
    
    def openbtnClicked(self):
        
        
        self.tuplefname = QFileDialog.getOpenFileName(self)
        self.fname = self.tuplefname[0]
        print(self.fname)
        dir,file = os.path.split(self.fname)
        print("1 ",dir, "에 있는 ", file)
#         상대경로로 쓰려고 할때 작업공간 계속 바꿔줬는데, 절대경로로 불러오면 안씀.
#   같은 위치 디렉토리 설정 에러 피하기 위해 동일한 디렉토리로는 변경하지 않음.
        self.dir1=dir
        if self.dir1==Left.dir1:
            None
        else:
            os.chdir(dir)
        Left.dir1=self.dir1
        
        print("현재 작업공간은 ",os.getcwd())
        self.label.setText(file)
        self.finame = file
        self.foname =os.path.basename(dir)
        self.flist = os.listdir(dir)
        print("2 ", self.flist)
        print("3 ", os.getcwd())
        
        #선택: 클릭한 확장자의 파일명을 클릭한 파일이 있는 폴더 내에서 모두 찾아 리스트 저장하기.(초기화 후)
        #선택: 클릭한 파일만 리스트에 저장하기(초기화 후)
        #텍스트브라우저에 나타내기
        search = os.path.splitext(file)[1]
        print("4 확장자는 ",search)
        
        self.sflist =[]
        self.tb.clear()
        if self.loadType == 0:
            
            
            for name in self.flist:
                if search in name:
                    self.sflist.append(name)
                    print("5 처리할 데이터 목록은 ", self.sflist)
                    self.tb.append(name)
                else:
                    None
            Left.sflist= self.sflist
                    
        else:
            self.sflist.append(file)
            print("5 처리할 데이터는 ", self.sflist)
            self.tb.append(file)
            
        Left.sflist = self.sflist
        Left.fname = self.fname
        Left.finame = self.finame
        #원래의 Graview내용
        print("Graview안에 있는 w,h는 ", self.w, self.h)
        Left.i = 0
        Left.changeView(self)
        
        
        
        
    def changeView(self):
        if Left.num ==0:
            None
        else:
            Left.deleteItems(self.middle)
#             self.view.deleteLater()
#             self.view = None
        
        self.view =CView(self)
        
        (Left.w,Left.h,Left.pixelsPerMetric,Left.width,Left.dB0)=ppp.ppp(Left.sflist[Left.i])
        
        Left.h = self.h
        Left.w = self.w
        
        
        self.view.setFixedHeight(self.h)
        self.view.setFixedWidth(self.w)
        self.middle.addWidget(self.view)
         # 전체 폼박스에 오른쪽 박스 배치
        self.formbox.addLayout(self.middle)

        self.formbox.setStretchFactor(self.middle, 0)
        print("이번에 띄울 이미지는 ",Left.i,"번째 이미지")
        self.setWindowTitle(self.sflist[Left.i])
        self.setGeometry(5,30,self.w+200,self.h)
        #self.showFullScreen()
        self.showMaximized()
        Left.scalepnt(self)
        self.lbl_2.setText(None)
        Left.num +=1
        
        self.lbl000="처리할 개수는 : "+str(len(self.sflist))+"개, "+"현재 "+str(Left.i+1)+"번째"
        self.lbl_000.setText(self.lbl000)
        self.lbl_000.adjustSize()
        
        self.lbl_3.setText("")
        self.lbl_3.adjustSize()
        global comment
        global ratio_pit
        global Rapid_angle_pit
        global angle_pit
        
        self.size=0
        self.dy_pit=0
        self.dx_pit=0
        ratio_pit=0
        Rapid_angle_pit=0
        angle_pit=0
        comment=""
        self.type=""
        
    
    def deleteItems(layout): 
        if layout is not None: 
            while layout.count(): 
                item = layout.takeAt(0)
                print(item)
                widget = item.widget() 

                if widget is not None: 
                    widget.deleteLater() 
                else: 
                    deleteItems(item.layout()) 

                    
    def previous_view(self):
        if Left.i ==0:
            None
        else: 
            Left.i-=1
        Left.changeView(self)
                    
                    
                
    def next_view(self):
        if Left.i ==len(self.sflist)-1:
            None
        else: 
            Left.i+=1
        Left.changeView(self)
    
    
    def lineEditChanged(self):
        Left.width=int(self.lineEdit.text())
        self.statusBar.showMessage(self.lineEdit.text()+" "+chr(956)+"m") 
    
    
    
    def scalepnt(self):
        global dy
#         global pixelsPerMetric
        
        
        self.txt_1=repr("{} ".format(Left.width)+chr(956)+"m")+"길이당 {:}".format(Left.dB0)+" pixels"    
        
        
        self.lbl_1.setText(self.txt_1)
        self.lbl_1.adjustSize()   
            
    def pixelpnt(self):
        global dy
        global comment
#         global pixelsPerMetric
        Left.pixelsPerMetric = Left.dB0 / Left.width
        print(Left.pixelsPerMetric)
        
        self.size= dy/Left.pixelsPerMetric
        
        self.txt_2="{}픽셀, ".format(dy)+repr("{:.5f} ".format(self.size)+chr(956)+"m")     
        
        if 'dx_pit' in globals():
            self.dx_pit=dx_pit/Left.pixelsPerMetric
            self.dy_pit=dy_pit/Left.pixelsPerMetric
        
        self.lbl_2.setText(self.txt_2)
        self.lbl_2.adjustSize()   
        
        if 'Rapid_angle_pit' in globals():
            if comment =="":
                if Rapid_angle_pit>91:#subsurface, undercutting
                    if Rapid_angle_pit>160:
                        self.type="d"
                    else:
                        self.type="e"
                elif ratio_pit<0.1:#super wide,Shallow(Custom)
                    self.type="cc"
                elif ratio_pit<0.5:#wide,Shallow
                    self.type="c"
                elif ratio_pit<1.1:#elliptical
                    self.type="b"
                else:#narrow.deep
                    self.type="a"
            else:
                self.type=""
        
        self.excelup()
        self.take_screenshot()
        #self.next_view()
        

        
    def take_screenshot(self):
        
        folder=self.foname+"_xlsx_Pitdepth_data"
        current_directory = os.getcwd()
        
        #디렉터리의 존재 여부를 확인하고, 없을 경우 해당 디렉터리를 생성
        
        try:
            if not(os.path.isdir(current_directory + os.sep+  folder)):
                os.makedirs(os.path.join(folder))
        except OSError as e:
            if e.errno != errno.EEXIST:
                print("Failed to create directory!!!!!")
                raise
        
        #screenshot코드 출처
        #https://stackoverflow.com/questions/51361674/is-there-a-way-to-take-screenshot-of-a-window-in-pyqt5-or-qt5
        scrname=Left.sflist[Left.i]
        p = QApplication.primaryScreen().grabWindow(self.view.winId())
        p.save(current_directory + os.sep+  folder + os.sep + self.foname + scrname, 'bmp')
        
        img = self.Image.open(current_directory + os.sep+  folder + os.sep + self.foname + scrname)
        
        #이미지텍스트  출처 https://codeday.me/ko/qa/20190315/69702.html
        #https://pillow.readthedocs.io/en/3.0.x/reference/ImageFont.html
        draw = self.ImageDraw.Draw(img)
        # font = ImageFont.truetype(<font-file>, <font-size>)
        font = self.ImageFont.truetype("arial.ttf", 40)#font =self.ImageFont.load_default()
        # draw.text((x, y),"Sample Text",(r,g,b))
        if comment =="x" or comment =="?":
            pass
        else:
            self.txt_10="Depth= "+repr("{:.5f} ".format(self.size)+chr(956)+"m")  
            draw.text((0, 0),self.txt_10,(0,0,0),font=font)
            if 'ratio_pit' in globals():
                self.txt_11="Ratio= "+repr("{:.5f} ".format(ratio_pit))  
                draw.text((0, 45),self.txt_11,(0,0,0),font=font)

            if 'Rapid_angle_pit' in globals():
                self.txt_12="Rapid Pit angle(deg)= "+repr("{:.0f} ".format(Rapid_angle_pit))+" Type:("+self.type+")"
                draw.text((0, 90),self.txt_12,(0,0,0),font=font)

            if 'angle_pit' in globals():
                self.txt_13="Angle(deg)= "+repr("{:.0f} ".format(angle_pit))  
                draw.text((0, 135),self.txt_13,(0,0,0),font=font)
        
        img.save(current_directory + os.sep+  folder + os.sep + self.foname + scrname)
        
        
        
        self.lbl_4.setText(" 완료 ")

    def excelup(self):
        global comment
        global ratio_pit
        global Rapid_angle_pit
        global angle_pit
        
        #https://docs.xlwings.org/en/stable/quickstart.html
        folder=self.foname+"_xlsx_Pitdepth_data"
        current_directory = os.getcwd()
        
        #디렉터리의 존재 여부를 확인하고, 없을 경우 해당 디렉터리를 생성
        
        try:
            if not(os.path.isdir(current_directory + os.sep+  folder)):
                os.makedirs(os.path.join(folder))
        except OSError as e:
            if e.errno != errno.EEXIST:
                print("Failed to create directory!!!!!")
                raise
        
        if os.path.isfile(current_directory + os.sep+  folder + os.sep + self.foname + "pit_depth_result.xlsx"):
            wb = xw.Book(current_directory + os.sep+  folder + os.sep + self.foname + "pit_depth_result.xlsx")
        else:
            wb = xw.Book()
        
        #이름이 있는 시트를 생성
        
        #(self.sflist[i],Left.width,self.size,self.foname)
        
        if comment =="x" or comment =="?":
            ratio_pit=0
            Rapid_angle_pit=0
            angle_pit=0
        else:
            pass
        #Sheet1에다 입력
        sht1 = wb.sheets[0]
        #sht1.range('A1').value = ['start', '','','Shape_factors -> ']
        sht1.range('A1').value = ['Data_name', 'scalebar ('+chr(956)+"m)", 'Pit_depth ('+chr(956)+"m)"\
                                  ,'Shape_depth_2 ('+chr(956)+"m)",'Shape_width_2 ('+chr(956)+"m)"]
        sht1.range('F1').value = ['Aspect_Ratio(H/L)','Rapid_Angle(deg)','Pit_Angle(deg)','Type(a,b,c,e)'\
                                  ,'comment','진짜시편번호','부식 깊이']  
        #parse추가(텍스트조합) 
        rn=parse("s({}_{})",self.foname)
        #print("8?=",rn.fixed[0],"22?=?",rn.fixed[1])
        rn1=rn.fixed[0]+"0"+rn.fixed[1]
        sht1.range('A2').value=[" "]
        sht1.range('K2').value=[rn1,"=MAX(OFFSET(Sheet1!$C$3,0,0,COUNTA(Sheet1!$C:$C)-1,1))"]
        self.exshadow = []
        self.rownum = sht1.range('A1').end('down').last_cell.row
        print('rownum',self.rownum)
        for i in range(self.rownum-1):
            print(sht1.range('A'+str(i+2)).value)
            self.exshadow.append(sht1.range('A'+str(i+2)).value)
        
        print('exshadow0',self.exshadow)
        if self.sflist[Left.i] not in self.exshadow:
            self.sfn=self.sflist[Left.i]
            self.exshadow.append(self.sfn)
            print('exshadow1',self.exshadow)
            
            sht1.range('A'+str(self.rownum+1)).value = [self.sflist[Left.i], Left.width, self.size , self.dy_pit\
                                                        ,self.dx_pit] 
            sht1.range('F'+str(self.rownum+1)).value = [ratio_pit, Rapid_angle_pit,angle_pit,self.type,comment]
        else:

            #exn=excel name
            exn=self.exshadow.index(self.sflist[Left.i])+1
            print('exe=',exn)
            sht1.range('A'+str(exn+1)).value = [self.sflist[Left.i], Left.width, self.size , self.dy_pit\
                                                        ,self.dx_pit]
            sht1.range('F'+str(exn+1)).value = [ratio_pit, Rapid_angle_pit,angle_pit,self.type,comment]
            print("type= ",self.type)
        #저장할 디렉토리를 알기위해, 현재의 디렉토리를 가져온다
        
        #현재디렉토리 아래의 xlsx_data라는 폴더에 작업한 엑셀파일을 저장
        #os.sep는 리눅스, 유니스 계열인 경우 슬래쉬, 윈도우 에서는 역슬래쉬 로 표시됨
        wb.save(current_directory + os.sep+  folder + os.sep + self.foname + "pit_depth_result.xlsx")
        #time.sleep(0.5)
#         wb.close()
        self.lbl_3.setText(" 완료 ")

        self.lbl_3.adjustSize()
        
        self.take_screenshot()


# QGraphicsView display QGraphicsScene
class CView(QGraphicsView):
    import math
    pics =[]
    num = 0
    itnum = 0
    def __init__(self, parent):
        
        super().__init__(parent)   
        
        self.dat= [Left.w,Left.h]
        print("CView로 받은 w,h 는 ",self.dat[:])
        
        print("CView에서의 작업공간은 ",os.getcwd())
        print("파일명은",Left.sflist[Left.i])
        
        
        self.scene = QGraphicsScene()
        

        pic = QPixmap('./'+Left.sflist[Left.i])

        self.scene.addPixmap(pic)
        self.setScene(self.scene)
        

        self.items = []
        self.items2 = []
        self.items3 =[]
        
        self.start = QPointF()
        self.end = QPointF()
        self.setRenderHint(QPainter.HighQualityAntialiasing)  
        
        CView.itnum=0
        space = 4;
        self.dashes = [9, space, 9, space, 9, space, 9, space, 9, space]
#     @staticmethod    
#     def getwhfname(w,h,fname):
        
#         return w, h, fname
        
    def moveEvent(self, e): # 없애면 스크롤바 생김
        rect = QRectF(self.rect())
        rect.adjust(0,0,-2,-2)

        self.scene.setSceneRect(rect)
    
    def mousePressEvent(self, e):
        
        if e.buttons():
#             if len(self.items) > 0:
#                 self.scene.removeItem(self.items[-1])
#                 del(self.items[-1])
        
#             if len(self.items2) > 0:
#                 self.scene.removeItem(self.items2[-1])
#                 del(self.items2[-1])
            pass
                
        if e.button() == Qt.RightButton:
            # 시작점 저장
            self.start = e.pos()
            self.end = e.pos() 
            CView.itnum += 1
            
        if e.button() == Qt.LeftButton:
            # 시작점 저장
            self.start = e.pos()
            self.end = e.pos() 
            
    
    def mouseMoveEvent(self, e):  
        
        if e.buttons() & Qt.RightButton:           
#             if len(self.items) > 0:
#                 self.scene.removeItem(self.items[-1])
#                 del(self.items[-1])  
            self.end = e.pos()
                #print("color= ",self.parent().pencolor)
                #https://wikidocs.net/37094
            pen = QPen(QColor(255,0,0), 2)
            
            global comment
            #Pit 그리기
                #print("itnum=",CView.itnum)
            if CView.itnum == 1:
                # 장면에 그려진 이전 선을 제거
                if len(self.items2) > 0:
                    self.scene.removeItem(self.items2[-1])
                    del(self.items2[-1]) 
                    
                     
                hori1 = QLineF(self.start.x(), self.start.y(), self.end.x(), self.start.y())

                self.items2.append(self.scene.addLine(hori1, pen))
            else:
                if len(self.items3) > 0:
                    self.scene.removeItem(self.items3[-1])
                    del(self.items3[-1])
                    self.scene.removeItem(self.items3[-1])
                    del(self.items3[-1])
                    self.scene.removeItem(self.items3[-1])
                    del(self.items3[-1])
                if self.restart_y-self.end.y() > 1:
                    if self.restart_xl-self.end.x() < 0 : 
                        a=QLineF(0,0,1000,500)
                        b=QLineF(1000,0,0,500)
                        c=QLineF(0,1,1,1)

                        self.items3.append(self.scene.addLine(a, pen))
                        self.items3.append(self.scene.addLine(b, pen))
                        self.items3.append(self.scene.addLine(c, pen))
                        comment = "x"
                    else:
                        a=QLineF(500,0,250,500)
                        b=QLineF(250,500,750,500)
                        c=QLineF(750,500,500,0)

                        self.items3.append(self.scene.addLine(a, pen))
                        self.items3.append(self.scene.addLine(b, pen))
                        self.items3.append(self.scene.addLine(c, pen))
                        comment = "?"
                        
                    
                else:
                    pen.setStyle(Qt.DashLine)
                    pen.setDashPattern(self.dashes)
                    pit = QLineF(self.end.x(), self.restart_y, self.end.x(), self.end.y())

                    pit_L = QLineF(self.restart_xl, self.restart_y, self.end.x(), self.end.y())

                    pit_R = QLineF(self.restart_xr, self.restart_y, self.end.x(), self.end.y())

                    self.items3.append(self.scene.addLine(pit, pen))
                    self.items3.append(self.scene.addLine(pit_L, pen))
                    self.items3.append(self.scene.addLine(pit_R, pen))
                CView.itnum==0
                
        # e.buttons()는 정수형 값을 리턴, e.button()은 move시 Qt.Nobutton 리턴 
        if e.buttons() & Qt.LeftButton:           

            self.end = e.pos()

            #pen = QPen(self.parent().pencolor, self.parent().combo.currentIndex())
            pen = QPen(QColor(0,0,0), 2)

        
            # 수평선 그리기
            if self.parent().drawType == 0:
                 
                # 장면에 그려진 이전 선을 제거            
                if len(self.items) > 0:
                    self.scene.removeItem(self.items[-1])
                    del(self.items[-1])  
                    self.scene.removeItem(self.items[-1])
                    del(self.items[-1])  
                
                hori1 = QLineF(0, self.start.y(), Left.w, self.start.y())
                
                self.items.append(self.scene.addLine(hori1, pen))
                
                hori2 = QLineF(self.end.x(), self.start.y(), self.end.x(), self.end.y())
                
                self.items.append(self.scene.addLine(hori2, pen))
                print(self.start.y(),self.end.y())
                
                
                
                """
                # 현재 선 추가
                hori = QLineF(self.start.x(), self.start.y(), self.end.x(), self.start.y())                
                self.items.append(self.scene.addLine(hori, pen))
                """

                
                
                
                
            
    
    def mouseReleaseEvent(self, e):  
        
        if e.button() == Qt.RightButton:
            
            global dx_pit
            global dy_pit
            global ratio_pit
            global Rapid_angle_pit
            global angle_pit
            
            
            if CView.itnum == 1:
                dx_pit = abs(self.start.x()-self.end.x())
                self.restart_y=self.start.y()
                self.restart_xl=min(self.start.x(),self.end.x())
                self.restart_xr=max(self.start.x(),self.end.x())
                print("restart_y=",self.restart_y)
            else:
                dy_pit = abs(self.restart_y-self.end.y())
                ratio_pit = dy_pit/dx_pit
                self.dxl=self.end.x()-self.restart_xl
                self.dxr=self.restart_xr-self.end.x()
                if self.dxl < 0:
                    self.inc_l= 180-math.degrees(math.atan(abs(self.restart_y-self.end.y())/abs(self.dxl)))
                    self.inc_r= math.degrees(math.atan(abs(self.restart_y-self.end.y())/self.dxr))
                    Rapid_angle_pit=round(self.inc_l)
                elif self.dxr < 0:
                    self.inc_l= math.degrees(math.atan(abs(self.restart_y-self.end.y())/self.dxl))
                    self.inc_r= 180-math.degrees(math.atan(abs(self.restart_y-self.end.y())/abs(self.dxr)))
                    Rapid_angle_pit=round(self.inc_r)
                elif self.dxl == 0:
                    self.inc_l=90
                    self.inc_r= math.degrees(math.atan(abs(self.restart_y-self.end.y())/abs(self.dxr)))
                    Rapid_angle_pit=90
                elif self.dxr == 0:
                    self.inc_l= math.degrees(math.atan(abs(self.restart_y-self.end.y())/self.dxl))
                    self.inc_r= 90
                    Rapid_angle_pit=90
                else:
                    self.inc_l= math.degrees(math.atan(abs(self.restart_y-self.end.y())/self.dxl))
                    self.inc_r= math.degrees(math.atan(abs(self.restart_y-self.end.y())/self.dxr))
                    Rapid_angle_pit=round(max(self.inc_l,self.inc_r))
                angle_pit=round(180-(self.inc_l+self.inc_r))
                #print(dx_pit ,dy_pit,ratio_pit,Rapid_angle_pit, angle_pit,ratio_pit)
                print(self.inc_l,self.inc_r,"rapid=",Rapid_angle_pit,"180?=",repr(self.inc_l+self.inc_r+angle_pit),"pit=",angle_pit)
                CView.itnum = 0

        if e.button() == Qt.LeftButton:

            
            global dy 
            dy = abs(self.start.y()-self.end.y())
            self.dx = abs(self.start.x()-self.end.x())
            print(dy, self.dx)
            

    """
            if self.parent().drawType == 0:
                
                self.items.clear()
                hori = QLineF(0, self.start.y(), w, self.start.y())
                self.scene.addLine(hori, pen)
                hori = QLineF(self.end.x(), self.start.y(), self.end.x(), self.end.y())
                self.scene.addLine(hori, pen)
                print(self.start,self.end)
    """  
    
    
class ppp():
    
    @staticmethod
    def midpoint(ptA, ptB):
        return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)  
    @staticmethod
    def ppp(fname):
    
       
        # global fname
    #     fname = './images/example_04.bmp'
        width=Left.width





        # load the image, convert it to grayscale, and blur it slightly
        image =cv2.imread(fname)
#         cv2.imshow('image',image)
        h, w = image.shape[:2]
        cv2.waitKey(0)
        h1=round(0.938*h)
        h2=round(0.97*h)
#         w2=round(0.691*w)
        w2=round(0.65*w)
        print(h1, h2, w2)
        image = image[h1:h2,0:w2]


        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        blur = cv2.GaussianBlur(gray, (7, 7), 0)

        # perform edge detection, then perform a dilation + erosion to
        # close gaps in between object edges
        canny = cv2.Canny(blur, 50, 100)
        edged = cv2.dilate(canny, None, iterations=1)
        edged = cv2.erode(edged, None, iterations=1)

        # find contours in the edge map
        cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        cnts = imutils.grab_contours(cnts)

        # sort the contours from left-to-right and initialize the
        # 'pixels per metric' calibration variable
        (cnts, _) = contours.sort_contours(cnts)
    #     global pixelsPerMetric
        pixelsPerMetric = None

        # loop over the contours individually
        for c in cnts:
            # if the contour is not sufficiently large, ignore it
            if cv2.contourArea(c) < 100:
                continue

            # compute the rotated bounding box of the contour
            orig = image.copy()
            box = cv2.minAreaRect(c)
            box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
            box = np.array(box, dtype="int")
            #print(box)

            # order the points in the contour such that they appear
            # in top-left, top-right, bottom-right, and bottom-left
            # order, then draw the outline of the rotated bounding
            # box
            box = perspective.order_points(box)
            cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)

            # loop over the original points and draw them
            for (x, y) in box:
                cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)

            # unpack the ordered bounding box, then compute the midpoint
            # between the top-left and top-right coordinates, followed by
            # the midpoint between bottom-left and bottom-right coordinates
            (tl, tr, br, bl) = box
            (tltrX, tltrY) = ppp.midpoint(tl, tr)
            (blbrX, blbrY) = ppp.midpoint(bl, br)

            # compute the midpoint between the top-left and top-right points,
            # followed by the midpoint between the top-righ and bottom-right
            (tlblX, tlblY) = ppp.midpoint(tl, bl)
            (trbrX, trbrY) = ppp.midpoint(tr, br)

            # draw the midpoints on the image
            cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
            cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)

            # draw lines between the midpoints
            cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
                (255, 0, 255), 2)
            cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
                (255, 0, 255), 2)

            # compute the Euclidean distance between the midpoints
            dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
            dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))

            
            if pixelsPerMetric is None:
                pixelsPerMetric = dB / width
                dB0=dB
            # compute the size of the object
            dimA = dA / pixelsPerMetric
            dimB = dB / pixelsPerMetric

            # draw the object sizes on the image
            cv2.putText(orig, "{:.5f}".format(dimB),
                (int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,0.65, (255, 255, 255), 2)
            cv2.putText(orig, "{:.5f}".format(dimA),(int(trbrX + 10), int(trbrY)),
                cv2.FONT_HERSHEY_SIMPLEX,0.65, (255, 255, 255), 2)   

            ##cv2.imshow("Image", orig)
            ##cv2.waitKey(0)
            ##cv2.destroyAllWindows()
        return w,h, pixelsPerMetric, width,dB0


if __name__=="__main__":
    app = QApplication(sys.argv)
    ex = Left()
    ex.show()
    sys.exit(app.exec_())
cv2.destroyAllWindows()
exit(0)

C:/Users/Hojin/Desktop/부식/전수조사 모음/전수조사 정리/Surfactant(X)_/30ml_/SP_/LiOH(0.06)_/INHIBIT(509)_/TEMP(200)_/HOUR(500)_/시연용/s(8_73)/8-73 down_15kV PC6 WD10_75.bmp
1  C:/Users/Hojin/Desktop/부식/전수조사 모음/전수조사 정리/Surfactant(X)_/30ml_/SP_/LiOH(0.06)_/INHIBIT(509)_/TEMP(200)_/HOUR(500)_/시연용/s(8_73) 에 있는  8-73 down_15kV PC6 WD10_75.bmp
현재 작업공간은  C:\Users\Hojin\Desktop\부식\전수조사 모음\전수조사 정리\Surfactant(X)_\30ml_\SP_\LiOH(0.06)_\INHIBIT(509)_\TEMP(200)_\HOUR(500)_\시연용\s(8_73)
2  ['8-73 down_15kV PC6 WD10_75.bmp', '8-73 down_15kV PC6 WD10_76.bmp', '8-73 down_15kV PC6 WD10_77.bmp', '8-73 down_15kV PC6 WD10_78.bmp', '8-73 down_15kV PC6 WD10_79.bmp', '8-73 down_15kV PC6 WD10_80.bmp', '8-73 down_15kV PC6 WD10_81.bmp', '8-73 down_15kV PC6 WD10_82.bmp', '8-73 down_15kV PC6 WD10_83.bmp', '8-73 down_15kV PC6 WD10_84.bmp', '8-73 down_15kV PC6 WD10_85.bmp', '8-73 down_15kV PC6 WD10_86.bmp', '8-73 down_15kV PC6 WD10_87.bmp', '8-73 up_15kV PC6 WD10_55.bmp', '8-73 up_15kV PC6 WD10_56.bmp', '8-73 up_15kV PC6 WD10_57.bm

CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\부식\전수조사 모음\전수조사 정리\Surfactant(X)_\30ml_\SP_\LiOH(0.06)_\INHIBIT(509)_\TEMP(200)_\HOUR(500)_\시연용\s(8_73)
파일명은 8-73 down_15kV PC6 WD10_76.bmp
961 993 832
이번에 띄울 이미지는  1 번째 이미지
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\부식\전수조사 모음\전수조사 정리\Surfactant(X)_\30ml_\SP_\LiOH(0.06)_\INHIBIT(509)_\TEMP(200)_\HOUR(500)_\시연용\s(8_73)
파일명은 8-73 down_15kV PC6 WD10_77.bmp
961 993 832
이번에 띄울 이미지는  2 번째 이미지
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\부식\전수조사 모음\전수조사 정리\Surfactant(X)_\30ml_\SP_\LiOH(0.06)_\INHIBIT(509)_\TEMP(200)_\HOUR(500)_\시연용\s(8_73)
파일명은 8-73 down_15kV PC6 WD10_78.bmp
961 993 832
이번에 띄울 이미지는  3 번째 이미지
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\부식\전수조사 모음\전수조사 정리\Surfactant(X)_\30ml_\SP_\LiOH(0.06)_\INHIBIT(509)_\TEMP(200)_\HOUR(500)_\시연용\s(8_73)
파일명은 8-73 down_15kV PC6 WD10_79.bmp
961 993 832
이번에 띄울 이미지는  4 번째 이미지
CView로 받은 w,h 는  [1280, 1024

CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\부식\전수조사 모음\전수조사 정리\Surfactant(X)_\30ml_\SP_\LiOH(0.06)_\INHIBIT(509)_\TEMP(200)_\HOUR(500)_\시연용\s(8_73)
파일명은 8-73 up_15kV PC6 WD10_61.bmp
961 993 832
이번에 띄울 이미지는  19 번째 이미지
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\부식\전수조사 모음\전수조사 정리\Surfactant(X)_\30ml_\SP_\LiOH(0.06)_\INHIBIT(509)_\TEMP(200)_\HOUR(500)_\시연용\s(8_73)
파일명은 8-73 up_15kV PC6 WD10_62.bmp
961 993 832
이번에 띄울 이미지는  20 번째 이미지
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\부식\전수조사 모음\전수조사 정리\Surfactant(X)_\30ml_\SP_\LiOH(0.06)_\INHIBIT(509)_\TEMP(200)_\HOUR(500)_\시연용\s(8_73)
파일명은 8-73 up_15kV PC6 WD10_61.bmp
961 993 832
이번에 띄울 이미지는  19 번째 이미지
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\부식\전수조사 모음\전수조사 정리\Surfactant(X)_\30ml_\SP_\LiOH(0.06)_\INHIBIT(509)_\TEMP(200)_\HOUR(500)_\시연용\s(8_73)
파일명은 8-73 up_15kV PC6 WD10_62.bmp
961 993 832
이번에 띄울 이미지는  20 번째 이미지
CView로 받은 w,h 는  [1280, 1024]
CV

CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\부식\전수조사 모음\전수조사 정리\Surfactant(X)_\30ml_\SP_\LiOH(0.06)_\INHIBIT(509)_\TEMP(200)_\HOUR(500)_\시연용\s(8_73)
파일명은 8-73 up_15kV PC6 WD10_65.bmp
961 993 832
이번에 띄울 이미지는  23 번째 이미지
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\부식\전수조사 모음\전수조사 정리\Surfactant(X)_\30ml_\SP_\LiOH(0.06)_\INHIBIT(509)_\TEMP(200)_\HOUR(500)_\시연용\s(8_73)
파일명은 8-73 up_15kV PC6 WD10_64.bmp
961 993 832
이번에 띄울 이미지는  22 번째 이미지
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\부식\전수조사 모음\전수조사 정리\Surfactant(X)_\30ml_\SP_\LiOH(0.06)_\INHIBIT(509)_\TEMP(200)_\HOUR(500)_\시연용\s(8_73)
파일명은 8-73 up_15kV PC6 WD10_63.bmp
961 993 832
이번에 띄울 이미지는  21 번째 이미지
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\부식\전수조사 모음\전수조사 정리\Surfactant(X)_\30ml_\SP_\LiOH(0.06)_\INHIBIT(509)_\TEMP(200)_\HOUR(500)_\시연용\s(8_73)
파일명은 8-73 up_15kV PC6 WD10_62.bmp
961 993 832
이번에 띄울 이미지는  20 번째 이미지
CView로 받은 w,h 는  [1280, 1024]
CV

390 527
390 527
390 527
390 527
390 527
390 527
390 527
390 527
390 526
390 526
390 526
390 526
390 525
390 525
390 525
390 525
390 525
390 525
390 525
390 525
390 524
390 524
390 524
390 523
390 523
390 523
390 523
390 523
390 523
390 523
390 523
390 524
134 348
444 444
444 444
444 444
444 444
444 444
0 5
restart_y= 444
0.0 90 rapid= 90 180?= 180.0 pit= 90
0.0 90 rapid= 90 180?= 180.0 pit= 90
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\부식\전수조사 모음\전수조사 정리\Surfactant(X)_\30ml_\SP_\LiOH(0.06)_\INHIBIT(509)_\TEMP(200)_\HOUR(500)_\시연용\s(8_73)
파일명은 8-73 down_15kV PC6 WD10_80.bmp
961 993 832
이번에 띄울 이미지는  5 번째 이미지
CView로 받은 w,h 는  [1280, 1024]
CView에서의 작업공간은  C:\Users\Hojin\Desktop\부식\전수조사 모음\전수조사 정리\Surfactant(X)_\30ml_\SP_\LiOH(0.06)_\INHIBIT(509)_\TEMP(200)_\HOUR(500)_\시연용\s(8_73)
파일명은 8-73 down_15kV PC6 WD10_81.bmp
961 993 832
이번에 띄울 이미지는  6 번째 이미지
395 395
395 395
395 395
395 395
395 395
395 395
395 395
395 395
395 395
395 395
395 395
395 395
395 395
395 395
395 3

337 390
337 391
337 392
337 394
337 395
337 396
337 397
337 398
337 399
337 401
337 403
337 404
337 406
337 408
337 408
337 411
337 412
337 415
337 416
337 417
337 419
337 421
337 423
337 424
337 426
337 429
337 431
337 433
337 435
337 437
337 438
337 441
337 443
337 446
337 446
337 449
337 451
337 454
337 455
337 456
337 458
337 459
337 461
337 463
337 464
337 465
337 466
337 467
337 469
337 470
337 471
337 472
337 474
337 475
337 477
337 478
337 480
337 481
337 482
337 484
337 486
337 487
337 488
337 490
337 492
337 492
337 494
337 495
337 496
337 498
337 499
337 500
337 501
337 502
337 504
337 505
337 506
337 508
337 510
337 512
337 513
337 515
337 517
337 518
337 520
337 521
337 524
337 525
337 527
337 530
337 532
337 534
337 536
337 538
337 540
337 542
337 543
337 545
337 547
337 548
337 550
337 551
337 553
337 554
337 555
337 557
337 558
337 559
337 561
337 562
337 563
337 564
337 565
337 566
337 567
337 568
337 569
337 571
337 571
337 572
337 573
337 573
337 575
337 576
337 576


AttributeError: 'NoneType' object has no attribute 'fixed'

SystemExit: 0

C:\Users\Hojin\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
